In [1]:
%cd ..
import os
import time

/home/youse/apperception


In [2]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

CUDA is available.
 > 0: Tesla T4
   1: Tesla T4


In [3]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
DATA_DIR = "/data/processed/full-dataset/trainval"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
RAW_DATA_DIR = "/data/raw/full-dataset/trainval"

In [43]:
import cv2
from os.path import exists
def construct_video(frames, size=(1600, 900), base=False, vid_name=None, vid_prefix=False):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig13_"+vid_name if vid_name else f"./outputs/{'fig13_base' if base else 'fig13_optimized'}.mp4"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        if vid_prefix:
            img_path = os.path.join(RAW_DATA_DIR, 'sweeps/CAM_FRONT', frame)
            if not exists(img_path):
                img_path = os.path.join(RAW_DATA_DIR, 'samples/CAM_FRONT', frame)
        else:
            img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [35]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
cam = camera

world = world.filter(
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck') | F.like(obj1.type, 'bus')) &
    F.contains_all('intersection', [obj1.trans]@cam.time)
)


In [36]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'], base=False,)

scene-0757-CAM_FRONT --------------------------------------------------------------------------------


100%|██████████| 8/8 [00:04<00:00,  1.69it/s]


Time taken to insert trajectories: 0.13284969329833984
num_video:  1
total preprocess time 25.192426443099976


In [37]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(1)
print(f'optimized query time: {time.time()-optimized_query_start}')

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        
        JOIN Cameras USING (cameraId)
        WHERE ((t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck' OR t0.objectType LIKE 'bus') AND (EXISTS(SELECT 1
            FROM SegmentPolygon
            WHERE
                SegmentPolygon.__RoadType__intersection__ AND
                ST_Covers(SegmentPolygon.elementPolygon, valueAtTimestamp(t0.translations,timestamp))
        )))
        
done execute node
Result length: 611
optimized query time: 0.1038503646850586


In [44]:
optimized_results = [e[3] for e in id_time_camId_filename]
construct_video(optimized_results)